Importy

In [5]:
import os
import pandas as pd

Wczytywanie danych

In [58]:
file_list = os.listdir("Dane")
func = lambda  a : a.replace(".txt", "").split("-")
count_size = lambda a : os.stat(a).st_size
data = [[func(title)[0],func(title)[1], "Dane/" + title, count_size("Dane/" + title)] for title in file_list]
df = pd.DataFrame(data, columns=["Author", "Name of The work", "FilePath", "FileSize"])

Przedstawiam zestaw uczący. Mamy tutaj pozycję kilku Polskich autorów.

In [64]:
print(df.drop(columns=["Name of The work", "FilePath"]).groupby("Author").sum())

                  FileSize
Author                    
AdamMickiewicz      761551
Juliusz Słowacki    329618


Implementacje tablicy mieszającej pobraliśmy ze źródła: https://www.geeksforgeeks.org/hash-map-in-python/

In [44]:
class HashTable:
    def __init__(self, size):
        self.size = size
        self.hash_table = [[] for _ in range(self.size)]

    def put(self, key, val):
        hashed_key = hash(key) % self.size
        bucket = self.hash_table[hashed_key]
        found_key = False
        for index, record in enumerate(bucket):
            record_key, record_val = record
            if record_key == key:
                found_key = True
                break
        if found_key:
            bucket[index] = (key, val)
        else:
            bucket.append((key, val))
            
    def get(self, key):
        hashed_key = hash(key) % self.size
        bucket = self.hash_table[hashed_key]
        found_key = False
        for index, record in enumerate(bucket):
            record_key, record_val = record
            if record_key == key:
                found_key = True
                break
        if found_key:
            return record_val
        else:
            return None

    def pop(self, key):
        hashed_key = hash(key) % self.size
        bucket = self.hash_table[hashed_key]
        found_key = False
        for index, record in enumerate(bucket):
            record_key, record_val = record

            if record_key == key:
                found_key = True
                break
        if found_key:
            bucket.pop(index)
            return record_val
        return None

Projekt ten używa worka słów, który to przechowuje informacje na temat częstości występowania słów w danym dokumencie. Definiujemy tu dwie funckje jedna do standaryzacji słów tzn. usunięcia ewentualnych znaków graficznych takich jak kropka czy przecinek oraz ustawienie wielkości liter na małe aby słowa "Który" oraz "który" były zliczane jako to same słowo.
Druga funkcja służy do budowania worka słów. Zlicza najpierw pojedyńcze przypadki wystąpień, a następnie zlicza częstość wystąpień danego słowa.

In [57]:
def normalize_word(word):
    word = word.lower()
    special_char = [".", ",", "-", "?", "(", ")", "!", "\\", "\"", ":", ";"]
    for char in special_char:
        word = word.replace(char, "")
    return word

def generate_word_bag(FileName):
    word_bag = HashTable(5000)
    over_all_words = 0
    every_word = []
    with open(FileName, "r") as f:
        for line in f:
            words = line.split()
            words = [normalize_word(word) for word in words]
            for word in words:
                if word != '':
                    over_all_words += 1
                    counter = word_bag.pop(word)
                    if counter != None:
                        word_bag.put(word, counter + 1)
                        every_word.append(word)
                    else:
                        counter = 0
                        word_bag.put(word, 1)
        f.close()
    for word in every_word:
        counter = word_bag.pop(word)
        word_bag.put(word, counter/over_all_words)
    return word_bag, every_word
        
file = "Dane/AdamMickiewicz-Burza.txt"
dictionary, words = generate_word_bag(file)
print("W pliku: \"", file,"\" Słowo: \"", words[4],"\" występowało z częstością: ", dictionary.get(words[4]), sep="")

W pliku: "Dane/AdamMickiewicz-Burza.txt" Słowo: "na" występowało z częstością: 0.0002883506343713956


Teraz musimy stworzyć model, który będzie klasyfikował do jakiego autora należy dany utwór.